In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
import torch
from pathlib import Path

# Config
MODEL_NAME = "Salesforce/codegen-350M-mono"

DATA_PATH = "./1-encoder-data/train/train.jsonl"
EVAL_PATH = "./1-encoder-data/eval/val.jsonl"
OUTPUT_DIR = "./finetuned-codegemma"

# Load dataset
train_data = load_dataset("json", data_files=DATA_PATH)["train"]
val_data = load_dataset("json", data_files=EVAL_PATH)["train"]

# Preprocess: concatenate input + output (zero-instruction)
def format_example(example):
    return {"text": example["input"] + "\n" + example["output"]}

train_data = train_data.map(format_example)
val_data = val_data.map(format_example)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token  # prevent pad token errors

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

train_data = train_data.map(tokenize, batched=True)
val_data = val_data.map(tokenize, batched=True)

# Load model
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
#for name, module in model.named_modules():
#    if "Linear" in str(type(module)):
#        print(name)


# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["qkv_proj", "out_proj", "fc_in", "fc_out"],  # Correct modules
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)


model = get_peft_model(model, lora_config)

# Training args
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=12,
    logging_steps=20,
    learning_rate=2e-4,
    fp16=True,
    report_to="none",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


last_checkpoint = None
checkpoint_dir = Path(OUTPUT_DIR)
checkpoints = list(checkpoint_dir.glob("checkpoint-*"))
if checkpoints:
    last_checkpoint = str(sorted(checkpoints, key=lambda p: int(p.name.split('-')[-1]))[-1])

# Start training (will resume if checkpoint exists)
trainer.train(resume_from_checkpoint=last_checkpoint)


# Train
#trainer.train()


c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.ut

{'loss': 0.2527, 'learning_rate': 6.659879775063022e-05, 'epoch': 8.0}


 67%|██████▋   | 13780/20628 [00:28<00:13, 513.47it/s] 

{'loss': 0.2615, 'learning_rate': 6.640488656195462e-05, 'epoch': 8.02}


 67%|██████▋   | 13800/20628 [00:47<01:19, 85.74it/s] 

...

                                                       
100%|██████████| 20628/20628 [1:53:45<00:00,  1.06it/s]

{'eval_loss': 0.3978927731513977, 'eval_runtime': 76.2653, 'eval_samples_per_second': 22.54, 'eval_steps_per_second': 11.276, 'epoch': 12.0}


100%|██████████| 20628/20628 [1:53:46<00:00,  3.02it/s]

{'train_runtime': 6826.0362, 'train_samples_per_second': 24.183, 'train_steps_per_second': 3.022, 'train_loss': 0.08490708646604159, 'epoch': 12.0}


TrainOutput(global_step=20628, training_loss=0.08490708646604159, metrics={'train_runtime': 6826.0362, 'train_samples_per_second': 24.183, 'train_steps_per_second': 3.022, 'train_loss': 0.08490708646604159, 'epoch': 12.0})

In [2]:
model.save_pretrained("./lora-adapter")
tokenizer.save_pretrained("./lora-adapter")


('./lora-adapter\\tokenizer_config.json',
 './lora-adapter\\special_tokens_map.json',
 './lora-adapter\\vocab.json',
 './lora-adapter\\merges.txt',
 './lora-adapter\\added_tokens.json',
 './lora-adapter\\tokenizer.json')